# Example of simple fit with PyTorch

* First implement in Numpy and with scipy.optimize
* Then try to do the same with PyTorch and TensorFlow
* Then try to do something more interesting with gamma-ray data

In [1]:
import numpy as np
from scipy.optimize import minimize

## Example data

Best-fit line has pars = (2, 0), i.e. slope 2, and intercept 0

In [2]:
x = np.array([1, 2, 3])
y = np.array([2, 4, 6])
data = {'x': x, 'y': y}

## Numpy & Scipy

Define cost function using Numpy; fit with scipy.optimize.minimize

In [3]:
def model(pars, x):
    return pars[0] * x + pars[1]

def cost(pars, data):
    y_model = model(pars, data['x'])
    dy = data['y'] - y_model
    return np.sum(dy ** 2)

pars = (42, 42) # any incorrect start values
res = minimize(cost, pars, args=(data,))
print(res) # correct result: slope = 2, intercept: 0

      fun: 5.602873721148142e-13
 hess_inv: array([[ 0.24995886, -0.50004917],
       [-0.50004917,  1.16660792]])
      jac: array([-5.37158973e-06, -2.43206103e-06])
  message: 'Optimization terminated successfully.'
     nfev: 32
      nit: 5
     njev: 8
   status: 0
  success: True
        x: array([ 1.99999984e+00, -9.94556075e-08])


## PyTorch

Let's try to do the same thing using PyTorch

In [4]:
import torch
from torch.autograd import Variable
from torch.optim import SGD

def model(x, w1, w2):
    return w1 * x + w2

def cost(out, label):
    return torch.sum((label - out) ** 2)

x = Variable(torch.Tensor([1, 2, 3]), requires_grad=False)
# x_tensor = Variable(torch.from_numpy(x))
y = Variable(torch.Tensor([2, 4, 6]), requires_grad=False)

w1 = Variable(torch.Tensor([42]), requires_grad=True) # any incorrect start values
w2 = Variable(torch.Tensor([42]), requires_grad=True) # any incorrect start values

optimizer = SGD([w1, w2], lr=0.01)

for i in range(1000):
    optimizer.zero_grad()

    y_model = model(x, w1, w2)
    loss = cost(y_model, y)
    loss.backward()
    optimizer.step()

print(float(w1))
print(float(w2))

1.9935346841812134
0.01469734963029623


## Tensorflow

Let's try to do the same thing using TensorFlow

In [6]:
import tensorflow as tf

x = tf.placeholder(tf.float32, [None, 1])
y_ = tf.placeholder(tf.float32, [None, 1])

b = tf.Variable(tf.zeros([1]))
w = tf.Variable(tf.zeros([1, 1]))

y = w * x + b

loss = tf.reduce_sum((y - y_) * (y - y_))
train_step = tf.train.GradientDescentOptimizer(0.005).minimize(loss)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for step in range(10):
    sess.run(train_step, feed_dict={x:[[2.3],[1.7],[-3.8],[0.5],[-4.1],[-1.5],[-2.5],[6.2]],
                                    y_:[[-4.4],[-3.6],[7.7],[-0.9],[8.3],[2.9],[4.9],[-12.2]]})
    print(step, sess.run(w), sess.run(b))

0 [[-1.7221999]] [0.02700001]
1 [[-1.9523064]] [0.03117361]
2 [[-1.9830445]] [0.03225204]
3 [[-1.9871444]] [0.03287534]
4 [[-1.9876854]] [0.03339959]
5 [[-1.9877515]] [0.0338754]
6 [[-1.9877547]] [0.03431235]
7 [[-1.9877499]] [0.03471432]
8 [[-1.9877445]] [0.03508417]
9 [[-1.9877392]] [0.03542452]
